In [1]:
!pip install comtypes


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import threading
import time
import datetime as dt
import json
import os
import logging
import gc
from pywifi import PyWiFi

In [3]:
FILE_PATH = 'rssi_data_test.json'

# Configure logging
logging.basicConfig(
    filename=f'rssi_{dt.datetime.now().strftime("%Y%m%d%H%M%S")}.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Threading control
stop_event = threading.Event()  # Used to stop the data collection thread
running = False  # Flag to indicate whether data collection is ongoing
crowd_count_lock = threading.Lock()  # Lock to handle concurrent access to crowd_count
crowd_count = None  # Variable to store the crowd count

def init(file_path):
    """Initialize the JSON file by creating an opening bracket for the array."""
    with open(file_path, 'w') as file:
        file.write('[')  # Start the JSON array
    logging.info(f"Initialized JSON file: {file_path}")

def append_record(file_path, record):
    """Append a single record to the JSON file without loading the entire file."""
    with open(file_path, 'a+') as file:
        file.seek(0, os.SEEK_END)
        # Add a comma before appending a new record (if it's not the first record)
        if file.tell() > 1:
            file.write(',')
        
        json.dump(record, file, indent=2)
        file.write('\n')
    logging.info(f"Appended new record to JSON file: {file_path}")

def finalize(file_path):
    """Add the closing bracket to finalize the JSON array."""
    with open(file_path, 'a') as file:
        file.write(']')  # End the JSON array
    logging.info(f"Finalized and closed JSON file: {file_path}")

def capture_rssi():
    """Capture RSSI data in a separate thread and handle stopping smoothly."""
    wifi = PyWiFi()
    iface = wifi.interfaces()[0]  # Assuming first interface is the one we want

    try:
        next_scan_time = time.time()  # Set initial scan time
        
        while not stop_event.is_set():
            start_time = time.time()

            iface.scan()  # Start a new scan
            results = iface.scan_results()

            # Get the current timestamp (without milliseconds)
            time_string = dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

            for network in results:
                # Safely access the crowd_count with a lock
                with crowd_count_lock:
                    current_crowd_count = crowd_count

                # Create a record to store in the JSON file
                record = {
                    'timestamp': time_string,  # Unique timestamp (without milliseconds)
                    'ssid': network.ssid,
                    'bssid': network.bssid,
                    'rssi': network.signal,
                    'crowd_count': current_crowd_count  # Include the crowd count
                }
                # Append the record to the JSON file
                append_record(FILE_PATH, record)

            # Force garbage collection to free up memory periodically
            gc.collect()

            # Print status to terminal
            print(f"Data collected at: {time_string} with crowd count: {current_crowd_count}")

            # Calculate the time for the next scan
            next_scan_time += 1.0
            sleep_duration = max(0, next_scan_time - time.time())  # Ensure non-negative sleep time
            time.sleep(sleep_duration)

    except Exception as e:
        print(f"Error during Wi-Fi scan: {e}")

# Function to start logging RSSI data
def start_logging():
    global running, capture_thread, crowd_count  # Ensure crowd_count is global
    if not running:
        # Take crowd count input if it's not set or if user wants to update it
        if crowd_count is None:
            crowd_count_input = input("Enter initial crowd count: ")
            with crowd_count_lock:
                crowd_count = crowd_count_input

        running = True
        stop_event.clear()  # Clear the stop event
        print(f"Data collection started with crowd count: {crowd_count}...")

        # Start the RSSI capture in a new thread
        capture_thread = threading.Thread(target=capture_rssi)
        capture_thread.start()

# Function to stop logging RSSI data
def stop_logging():
    global running
    if running:
        running = False
        stop_event.set()  # Signal the thread to stop
        print("Stopping data collection...")

        # Wait for the thread to finish
        capture_thread.join()

        # Finalize the JSON file
        finalize(FILE_PATH)
        print("Data collection stopped. Ready for next command.")

def main():
    global crowd_count
    # Initialize the JSON file
    init(FILE_PATH)

    while True:
        command = input("Enter 'Y' to start logging, 'N' to stop logging, 'X' to quit, or provide new crowd count: ").upper()

        if command == 'Y':
            start_logging()
        elif command == 'N':
            if running:
                stop_logging()
            else:
                print("Data collection is not running.")
        elif command == 'X':
            if running:
                stop_logging()
            print("Exiting the program.")
            break
        else:
            # If the command is not 'Y', 'N', or 'X', assume it's a new crowd count
            try:
                new_crowd_count = int(command)  # Set new crowd count
                with crowd_count_lock:
                    crowd_count = new_crowd_count  # Update the crowd count
                print(f"Crowd count updated to: {crowd_count}")
            except ValueError:
                print("Invalid input. Please enter 'Y', 'N', 'X', or a valid crowd count.")

if __name__ == '__main__':
    main()

Enter 'Y' to start logging, 'N' to stop logging, 'X' to quit, or provide new crowd count:  y
Enter initial crowd count:  5


Data collection started with crowd count: 5...
Data collected at: 2024-10-18 23:24:25 with crowd count: 5
Data collected at: 2024-10-18 23:24:26 with crowd count: 5
Data collected at: 2024-10-18 23:24:27 with crowd count: 5
Data collected at: 2024-10-18 23:24:28 with crowd count: 5
Data collected at: 2024-10-18 23:24:29 with crowd count: 5


Enter 'Y' to start logging, 'N' to stop logging, 'X' to quit, or provide new crowd count:  x


Stopping data collection...
Data collection stopped. Ready for next command.
Exiting the program.
